In [ ]:
!git clone https://github.com/ggerganov/llama.cpp

Cloning into 'llama.cpp'...
remote: Enumerating objects: 19651, done.
remote: Counting objects: 100% (6999/6999), done.
remote: Compressing objects: 100% (549/549), done.
remote: Total 19651 (delta 6752), reused 6535 (delta 6447), pack-reused 12652
Receiving objects: 100% (19651/19651), 23.64 MiB | 21.16 MiB/s, done.
Resolving deltas: 100% (13897/13897), done.


In [ ]:
!cd llama.cpp && LLAMA_CUBLAS=1 make && pip install -r requirements.txt

I ccache not found. Consider installing it for faster compilation.
I llama.cpp build info: 
I UNAME_S:   Linux
I UNAME_P:   x86_64
I UNAME_M:   x86_64
I CFLAGS:    -I. -Icommon -D_XOPEN_SOURCE=600 -D_GNU_SOURCE -DNDEBUG -DGGML_USE_CUBLAS -I/usr/local/cuda/include -I/usr/local/cuda/targets/x86_64-linux/include  -std=c11   -fPIC -O3 -Wall -Wextra -Wpedantic -Wcast-qual -Wno-unused-function -Wshadow -Wstrict-prototypes -Wpointer-arith -Wmissing-prototypes -Werror=implicit-int -Werror=implicit-function-declaration -pthread -march=native -mtune=native -Wdouble-promotion 
I CXXFLAGS:  -std=c++11 -fPIC -O3 -Wall -Wextra -Wpedantic -Wcast-qual -Wno-unused-function -Wmissing-declarations -Wmissing-noreturn -pthread  -march=native -mtune=native -Wno-array-bounds -Wno-format-truncation -Wextra-semi -I. -Icommon -D_XOPEN_SOURCE=600 -D_GNU_SOURCE -DNDEBUG -DGGML_USE_CUBLAS -I/usr/local/cuda/include -I/usr/local/cuda/targets/x86_64-linux/include 
I NVCCFLAGS: -std=c++11 -O3 -use_fast_math --forward-

In [ ]:
from huggingface_hub import snapshot_download

In [ ]:
model_name = "Qwen/Qwen1.5-1.8B"

In [ ]:
methods = ['q4_k_m']

In [ ]:
base_model = "./original_model/"
quantized_path = "./quantized_model/"

In [ ]:
snapshot_download(repo_id=model_name, local_dir=base_model , local_dir_use_symlinks=False)
original_model = quantized_path+'/FP16.gguf'

Fetching 10 files:   0%|          | 0/10 [00:00<?, ?it/s]

README.md:   0%|          | 0.00/2.71k [00:00<?, ?B/s]

.gitattributes:   0%|          | 0.00/1.52k [00:00<?, ?B/s]

LICENSE:   0%|          | 0.00/7.28k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

...

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.16k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.67G [00:00<?, ?B/s]

In [ ]:
!mkdir ./quantized_model/

In [ ]:
!python llama.cpp/convert-hf-to-gguf.py ./original_model/ --outtype f16 --outfile ./quantized_model/FP16.gguf

Loading model: original_model
gguf: This GGUF file is for Little Endian only
Set model parameters
Set model tokenizer
The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.
0it [00:00, ?it/s]
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
gguf: Adding 151387 merge(s).
gguf: Setting special token type eos to 151643
gguf: Setting special token type pad to 151643
gguf: Setting special token type bos to 151643
gguf: Setting chat_template to {% for message in messages %}{{'<|im_start|>' + message['role'] + '
' + message['content'] + '<|im_end|>' + '
'}}{% endfor %}{% if add_generation_prompt %}{{ '<|im_start|>assistant
' }}{% endif %}
Exporting model to 'quantized_model/FP16.gguf'
gguf: loading model part 'model.safetensors'
output.weight, n_dims 

In [ ]:
import os

In [ ]:
for m in methods:
    qtype = f"{quantized_path}/{m.upper()}.gguf"
    os.system("./llama.cpp/quantize "+quantized_path+"/FP16.gguf "+qtype+" "+m)

In [ ]:
! ./llama.cpp/main -m ./quantized_model/Q4_K_M.gguf -n 90 --repeat_penalty 1.0 --color -i -r "User:" -f llama.cpp/prompts/chat-with-bob.txt

Log start
main: build = 2333 (a0fc6266)
main: built with cc (Ubuntu 11.4.0-1ubuntu1~22.04) 11.4.0 for x86_64-linux-gnu
main: seed  = 1709552864
ggml_init_cublas: GGML_CUDA_FORCE_MMQ:   no
ggml_init_cublas: CUDA_USE_TENSOR_CORES: yes
ggml_init_cublas: found 1 CUDA devices:
  Device 0: Tesla T4, compute capability 7.5, VMM: yes
llama_model_loader: loaded meta data with 20 key-value pairs and 291 tensors from ./quantized_model/Q4_K_M.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = qwen2
llama_model_loader: - kv   1:                               general.name str              = original_model
llama_model_loader: - kv   2:                          qwen2.block_count u32              = 24
llama_model_loader: - kv   3:                       qwen2.context_length u32              = 32768
llama_model_loader: - kv   4:        

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
from huggingface_hub import HfApi, HfFolder, create_repo, upload_file

In [ ]:
# model_path = "/content/quantized_model/Q4_K_M.gguf" # Your model's local path
# repo_name = "qwen1.5-8b-llm-gguf"  # Desired HF Hub repository name
# repo_url = create_repo(repo_name, private=False)

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
from huggingface_hub import HfApi, HfFolder, create_repo, upload_file

model_path = "/content/quantized_model/Q4_K_M.gguf" # Your model's local path
write_token = ""  # Replace "your_write_token_here" with your actual write token

api = HfApi()
api.upload_file(
    path_or_fileobj=model_path,
    path_in_repo="Q4_K_M.gguf",
    repo_id="asad1499/qwen1.5-8b-llm-gguf",
    repo_type="model",
    use_auth_token=write_token  # Pass the write token here
)


Q4_K_M.gguf:   0%|          | 0.00/1.22G [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/asad1499/qwen1.5-8b-llm-gguf/commit/bac485eb527d410680a7d47f83464398eb06985e', commit_message='Upload Q4_K_M.gguf with huggingface_hub', commit_description='', oid='bac485eb527d410680a7d47f83464398eb06985e', pr_url=None, pr_revision=None, pr_num=None)